# UTILITIES:

In [ ]:
! pip install bertopic -q
! pip install langchain -q
! pip install sentence-transformers chromadb openai -q

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import numpy as np
import pandas as pd
import re
from pprint import pprint as pp

# BERTOPIC:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

# LANCHAIN, CHROMADB & OPENAI:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

import openai
from openai import OpenAI

## BERTOPIC FUNCS:

In [ ]:
def f_make_text_chunks(raw_txt):

    txt = raw_txt.replace('\ufeff', '')
    txt_chunks = re.split('\.\s*\n', txt)
    txt_chunks = [txt_chunk.replace('\n', '') for txt_chunk in txt_chunks]
    return txt_chunks


def f_get_topics_dict(text_chunks, topic_model, ner_model, n_top = 10):

    df_docs = topic_model.get_document_info(text_chunks)

    topic_dict = {}
    for i in topic_model.get_topics().keys():

        filter = df_docs.Topic == i
        df_top_docs = df_docs[filter].sort_values(by = 'Probability', ascending = False)
        df_top_docs = df_top_docs.iloc[:n_top].Document

        keywords = topic_model.get_topic(i)
        keywords = [pair[0] for pair in keywords]
        keywords = set([i.lower() for i in keywords])

        ner_words = ner_model(list(df_top_docs.values))
        ner_words_ = [i[0] for i in ner_words if len(i) > 0]
        ner_words_ = set([i['word'] for i in ner_words_])
        ner_words_ = set([i.lower() for i in ner_words_])

        topic_words = list(ner_words_ | keywords)
        topic_dict[i] = topic_words

    return topic_dict



def f_display_topics(topic_dict):

    for i in topic_dict:
        print(f'TOPIC: {i}')
        print('---------')
        topics = topic_dict[i]
        topics = ", ".join(topics)
        pp(topics)
        print()
        print()




def f_get_topic_docs(topic_n, topic_model):

    df_topics = topic_model.get_document_info(txt_chunks)
    df_topics.columns = [i.lower() for i in df_topics.columns]

    df_representative = (df_topics
    [df_topics.topic == topic_n]
    [df_topics.representative_document == True]
    .sort_values(by = 'probability', ascending = False)
    .document)

    df_not_representative = (df_topics
    [df_topics.topic == topic_n]
    [df_topics.representative_document == False]
    .sort_values(by = 'probability', ascending = False)
    .document)

    df_topic_docs = (pd.concat
     ([df_representative, df_not_representative])
    .reset_index(drop = True))

    return df_topic_docs




def f_explore_topic_docs(topic_n, topic_model, ner_model, txt_chunks, n_samples = 3):

    topic_dict = f_get_topics_dict(txt_chunks, topic_model, ner_model, n_top = 10)
    df_top_docs = f_get_topic_docs(topic_n, topic_model)

    print('TOPIC KEYWORDS:')
    words = topic_dict[topic_n]
    words = ", ".join(words)
    pp(words)
    print()

    n_samples = n_samples if len(df_top_docs) >= n_samples else len(df_top_docs)

    top_docs = df_top_docs.iloc[:n_samples]
    for doc in top_docs:
        print('------------')
        pp(doc)
        print()

## TOPIC MODEL:

In [ ]:
def f_fit_topic_model(txt_chunks, n_neighs_umap = 3, n_dims_umap = 10,
                   min_cluster_size = 3, dist_metric = 'euclidean'):

    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

    # Step 2 - Reduce dimensionality
    umap_model = UMAP(n_neighbors=n_neighs_umap,
                    n_components=n_dims_umap,
                    random_state=42,
                    min_dist=0.0,
                    metric='cosine')

    # Step 3 - Cluster reduced embeddings
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size,
                            metric=dist_metric,
                            cluster_selection_method='eom',
                            prediction_data=True)

    # Step 4 - Tokenize topics
    vectorizer_model = CountVectorizer(stop_words="english")

    # Step 5 - Create topic representation
    ctfidf_model = ClassTfidfTransformer()

    # Step 6 - (Optional) Fine-tune topic representations with
    # a `bertopic.representation` model
    representation_model = KeyBERTInspired()

    # All steps together
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        representation_model=representation_model
    )

    topics, probs = topic_model.fit_transform(txt_chunks)

    ner_model = pipeline("token-classification",
                          model="wizardofchance/ner-model",
                          aggregation_strategy="simple")

    topic_dict = f_get_topics_dict(txt_chunks, topic_model, ner_model, n_top = 10)

    return topic_model, ner_model, topic_dict

## RAG

In [ ]:
def f_chroma_instantiations():

    embedding_function = SentenceTransformerEmbeddingFunction()

    character_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=1000,
        chunk_overlap=0)

    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0,
                                                        tokens_per_chunk=256)

    return embedding_function, character_splitter, token_splitter



def f_chroma_input(topic_n, topic_model, character_splitter, token_splitter):

    topic_docs = f_get_topic_docs(topic_n, topic_model).to_list()
    topic_doc_consolidated  = '\n\n'.join(topic_docs)

    topic_chunks = character_splitter.split_text(topic_doc_consolidated)
    chroma_input_docs = []
    for chunk in topic_chunks:
        chroma_input_docs += token_splitter.split_text(chunk)

    return chroma_input_docs




def f_create_chroma_collection(topic_n, topic_model, collection_name,
                               chroma_client, embedding_function,
                               character_splitter, token_splitter):

    collection = chroma_client.create_collection(
                        collection_name,
                        embedding_function=embedding_function)



    # Add documents to the collection
    chroma_input_docs = f_chroma_input(topic_n, topic_model,
                                       character_splitter, token_splitter)
    ids = [str(i) for i in range(len(chroma_input_docs))]
    collection.add(ids=ids, documents=chroma_input_docs)

    return collection




def f_create_chroma_collections(topic_dict, topic_model,
                               chroma_client, embedding_function,
                               character_splitter, token_splitter):

    for i in chroma_client.list_collections():
        chroma_client.delete_collection(i.name)

    for topic_n in topic_dict:

        collection_name = f'topic_{topic_n}'

        f_create_chroma_collection(topic_n, topic_model, collection_name,
                                    chroma_client, embedding_function,
                                    character_splitter, token_splitter)
        print(f"""collection '{collection_name}' created sucessfully""")






def rag(query, prompt, retrieved_documents, openai_client, llm):

    information = "\n\n".join(retrieved_documents)
    user_content = f"Question: {query}. \n Information: {information}"

    messages = [{"role": "system", "content": prompt},
                {"role": "user", "content": user_content}]

    response = openai_client.chat.completions.create(
        model=llm,
        messages=messages)

    content = response.choices[0].message.content
    return content




def f_get_ans(query, topic_n, prompt, openai_client, llm, n_results = 10):

    collection_name = f'topic_{topic_n}'
    chroma_collection = chroma_client.get_collection(collection_name)

    results = chroma_collection.query(query_texts=[query], n_results=n_results)
    retrieved_documents = results['documents'][0]
    output = rag(query, prompt, retrieved_documents, openai_client, llm)

    print()
    print('ANSWER: ')
    print('--------')
    print(output)

***

# A. TEXT DOC:
(COPY & PASTE)

In [ ]:
txt_doc = """\
The United Nations is marking its seventieth anniversary this year, making this session of the General Assembly a historic one. I hope that this year will also be historic for the United Nations in terms of outcomes. I would like to assure the President that he will receive India’s full support in his efforts.
Seventy years ago, the foundations of the United Nations were laid at the San Francisco Conference, in a city on the west coast of this country. India was one of the signatory countries of the Charter of the United Nations, although at that time we were not independent. We obtained our independence two years later. When the United Nations was established, a rather diminutive-looking man with the powerful weapon of non-violence was writing the final act in a struggle that would become a symbol of hope for the colonized and the oppressed everywhere. I am grateful that the United Nations has proclaimed the birthday of this extraordinary man as the International Day of Non-Violence. It is a pleasing coincidence that tomorrow is 2 October, and
42/55 15-29658


that we will be celebrating his birthday then. Mahatma Gandhi said, “The difference between what we do and what we are capable of doing would suffice to solve most of the world’s problems”. His message is most fitting in the context of the United Nations.
The seventieth year of any human being’s life is a milestone from which to look back and reflect on what one has achieved and what one has lost. Similarly, for people associated with an institution, the seventieth year provides an opportunity to analyse whether the institution has fulfilled its purpose and achieved the goals set for it at its establishment. Today we need to ask ourselves whether we have fulfilled the purpose and achieved the goals for which the United Nations was established 70 years ago. When I ask myself that question, I receive an affirmative answer for some questions and a negative one for others. For example, the United Nations has been successful in preventing a third world war, assisting decolonization and dismantling apartheid. It has been successful in combating global epidemics, reducing global hunger and promoting democracy and human rights.
Yet, when we ask ourselves whether we have been able to prevent conflicts taking place in various parts of the world, the answer is “no”. If we ask whether we were able to find permanent solutions to those conflicts, the answer is “no”. If we ask whether we were able to show the path of peace to a world which is treading the path of violence, the answer is “no”. According to those parameters, the United Nations appears to be ineffective at maintaining international peace and security, an institution that has failed to effectively address new challenges in that area.
Today, the world is ravaged by war on three continents, with the Security Council powerless or unwilling to stem the f low of blood. Traditional solutions that rely on force have only exacerbated problems. We must ask ourselves if we have the political will to craft alternatives to conflict and to pursue them with commitment and single-minded dedication. Nowhere is such a goal more important than in peacekeeping. Under the blue flag, men and women are constantly working to prevent conflict, protect civilians and sustain peace processes. With 180,000 peacekeepers deployed so far, India has been the largest contributor to international security provided by the United Nations. Even today, about 8,000 Indian military and police personnel are participating in 10 peacekeeping missions, operating in highly challenging environments.
India remains committed to continuing its support to United Nations peacekeeping operations and even to enhancing its contributions, as announced by our Prime Minister at the leaders summit on peacekeeping. Our new contributions will cover all aspects of peacekeeping: personnel, enablers and training. At the same time, no dilution of the cardinal principles of peacekeeping must occur. The fact that troop-contributing countries have no role in the formulation of mandates, which are often amended without consultation, is a matter of concern. That is a clear violation of Article 44 of the Charter of the United Nations. We also believe that peacekeeping operations cannot be a substitute for political solutions, a fact that the High-Level Independent Panel on Peace Operations underscored.
As we mark the seventieth anniversary of the United Nations, I take this opportunity to pay tribute to the more than 3,300 peacekeepers, including 161 from India, who have made the ultimate sacrifice. We stand ready to contribute to the memorial wall for United Nations peacekeepers, which was approved by the General Assembly at its sixty-ninth session.
The safety of our peacekeepers, the security of our nations, and indeed the future of the international community itself now depend on how we respond to the greatest threat that we face today: terrorism. It is a threat that India has lived with for more than a quarter of a century, and one that was tragically brought home to this very city in the autumn of 2001. Since then, we have not adequately countered the proliferation of terrorist acts, the rise of extremist ideologies and the impunity of States that back terrorism.
Only organized international action can defeat international terrorism. The world must demonstrate zero tolerance for terrorists who kill and maim innocent civilians and base its action on the principle of prosecute or extradite. The international community must make countries that provide financing to terrorists and safe havens for their training, arming and operations pay a heavy price.
Equally important, the establishment of an international legal regime, under a comprehensive convention on international terrorism, can no longer be held up. Nineteen years ago, in 1996, India proposed such a regime at the United Nations, but the Member States have been unable to adopt it and have become mired in the issue of definition. We have to understand that no such distinction exists as good terrorists and bad terrorists, and that terrorism cannot be linked
15-29658 43/55

to any religion. A terrorist is a terrorist, and anyone who commits crimes against humanity cannot have a religion. Therefore, I appeal to everyone to come together during this seventieth anniversary year of the United Nations and to pledge to unanimously adopt a comprehensive convention on international terrorism.
While on the subject of terrorism, I take this opportunity to share the challenges that we face in our ties with Pakistan. None of us can accept that terrorism is a legitimate instrument of statecraft. The world shared our outrage at the 2008 Mumbai terror attacks, in which citizens of many nations were helplessly butchered. That the mastermind behind the attack is walking free is an affront to the entire international community. Not only have past assurances in this regard not been honoured, but new cross-border terrorist attacks have taken place recently, and two terrorists from across the border have been captured alive. We all know that these attacks are meant to destabilize India and legitimize Pakistan’s illegal occupation of parts of the Indian State of Jammu and Kashmir and its claim to the rest of it.
Let me use this occasion to spell out our approach clearly. India remains open to dialogue but finds talks and terrorism to be incompatible. Yesterday, the Prime Minister of Pakistan proposed what he termed a new four-point peace initiative. I would like to respond to this. We do not need four points. We need just one — give up terrorism and let us sit down and talk. This will solve all the problems. That was precisely what was discussed and decided by the two Prime Ministers in Ufa this July. Let us hold talks, at the level of national security advisers, on all issues connected to terrorism and arrange an early meeting of our Directors-General of Military Operations to address the situation on the border. If the response is serious and credible, India is prepared to address all outstanding issues through bilateral dialogue. Even as we counter the menace of terrorism, we must acknowledge that real social and economic progress remains a critical goal. The elimination of basic human want leads almost invariably to more peaceful societies, as is evidenced by a map of the conflicts that engulf parts of the world.
Just a few days ago, we adopted the 2030 Agenda for Sustainable Development (resolution 70/1), with the Sustainable Development Goals at its core. They constitute a blueprint that is more comprehensive and holistic than the Millennium Development Goals. However, their successful implementation will require political resolve and an inclination to
share both technology and financial resources, while simultaneously improving our delivery mechanisms. As a woman and an elected Member of Parliament, it has been my firm conviction that there is a shortcut to real social change, and that is empowering the girl child. Our Government’s policy and our Prime Minister’s programme of Educate the Girl Child to Empower Her was conceived with that vision.
Our future rests on our building a sustainable planet for our children and our children’s children. As we prepare to meet in two months’ time, in Paris, the world expects us to deliver an ambitious and credible agreement on climate change. We have a duty to take common action, but in doing so we must keep in mind the larger historical contributions of some and the differentiated responsibilities of others. If Mahatma Gandhi were among us today, he would ask if we have used the resources of the planet for our needs or for our greed. He would also ask if adapting our lifestyle choices and reducing extravagant consumption would help us to correct our course. For this reason, the agreement in Paris needs to be comprehensive and equitable, while delivering concrete action. Developing countries can do more if they are enabled in their efforts by support from developed countries in the areas of finance, technology transfer and capacity- building. India is and always has been a willing partner in global efforts towards this goal. We will play our due role in reaching a meaningful, equitable and effective agreement in Paris.
India’s reverence for the environment is based on our traditions, which have always held nature as sacred. The health of the planet is tied to our own well-being, and it is this holistic approach that guided us in our call for the First International Day of Yoga, which was celebrated with much fervour in 192 countries of the world, as well as in this hallowed institution. I thank the international community for its enthusiasm and support in this regard. I hope that the seed that has been planted will grow into a magnificent tree.
I also congratulate States for their prompt action in tackling the threat of global epidemics. By linking our efforts, we have managed to eradicate smallpox and nearly eradicate polio. HIV/AIDS has been controlled and, more recently, the Ebola outbreak was contained, although not before an unacceptably high price in lives had been paid. The Ebola epidemic is an important example of how national action and international support can resolve a global crisis. We are proud that
44/55 15-29658


India played its due part in these efforts. Inspired by our successes, we must redouble our efforts to eradicate other diseases, whether it be tuberculosis or malaria.
Natural disasters and man-made conflicts also deserve our coordinated response. In recent months, the international community has been found wanting in its response to the refugee crisis caused by the conflict situations in Syria, Iraq and Libya. What is needed is a swift response backed by political will. Our own response to humanitarian crises in our neighbourhood has been quick, responsive and holistic. Whether in Nepal or Yemen, India has emerged as a net security provider, assisting not only our nationals but those of other countries that sought our help. India will be hosting the first Asian Ministerial Conference on Disaster Risk Reduction in November 2016, when the topic will be the Sendai Framework for Disaster Risk Reduction.
Like the United Nations, India has walked a long way these past 70 years. As 1.25 billion of my countrymen would attest, we have done so happy in the knowledge that our democratic traditions are deeply embedded. Nonetheless, every once in a while, there is a need for transformative change — one that can reinvigorate the nation and redeem its spirit. Ever since the new Government of Prime Minister Modi took office a year and a half ago, we have committed ourselves to this renewal. It is one that has enabled India to play its rightful role in international affairs, while also becoming the fastest-growing major economy in the world. The driving force of this comprehensive renewal has been premised on the motto of sabka saath, sabka vikas, which places emphasis on the welfare of all our people.
At the same time, we have started to reach out enthusiastically to all our partners in the international community. Our efforts have focused, first and foremost, on our immediate neighbourhood and the results have been truly transformational. We have also reached out to our extended neighbourhood, renewing ancient linkages and constructing modern economic partnerships, including, most recently, a summit with 14 Pacific island nations. Our Act East policy has replaced the previous one, Look East, with a more vigorous and proactive engagement in an economically vibrant region, and complements our Link West engagement.
We remain committed to the Middle East peace process, which is key to preventing further radicalization of the region. We continue to hope that a
solution can be reached to the Palestinian question. We have also qualitatively upgraded our relations with all the major Powers.
Africa is a region with which we have historical bonds, a solidarity born out of a common struggle against colonialism and a belief in a future of shared prosperity. Later this month we will host the third India- Africa Forum Summit, the first time that the leaders of all 54 African countries have been invited.
As a nuclear-weapon State, India is aware of its responsibility, and its support for global, non-discriminatory and verifiable nuclear disarmament is undiminished.
As with nations, so it is with institutions. Only periodic renewal can provide an organization with meaning and purpose. In a world that continues to be dominated by wealthy and influential nations, the United Nations notion of sovereign equality has permitted the developing world to question some unfair norms. But it has not permitted a fundamental challenge to the inequity of a system built for a world that longer exists. If we are to preserve the centrality and legitimacy of the United Nations as the custodian of global peace, security and development, the most urgent and pressing need is reform of the Security Council. This is the need of the hour. How can we have a Security Council in 2015 that still reflects the geopolitical architecture of 1945? How can we have a Security Council that still does not give Africa and Latin America seats as permanent members?
We must include more developing nations in the decision-making structures of the Security Council. And we must change the way it does business by doing away with outdated and non-transparent working methods. Giving it more legitimacy and balance would restore its credibility and equip it to confront the challenges of our times. We are happy that during the past year, under the leadership of Their Excellences Mr. Sam Kutesa and Mr. Courtenay Rattray, we have achieved what we had been unable to do in two decades of discussions — a text for negotiations unanimously adopted by the General Assembly as decision 69/560. That first but critical step must be a springboard for action at this historic seventieth session.
For an organization such as the United Nations, 70 years has a unique significance. It is an occasion for revitalization and for renewal. I invite everyone in this Hall to contemplate the idea of the United Nations as
15-29658 45/55


a banyan tree. In the eastern tradition, the banyan tree connotes wisdom, and yet is non-judgemental and all- encompassing. It has not one trunk but many, growing outwards, each connected to the others as well as to the sky and the earth. Its expansive branches provide shade and relief, and its base has always been a place for discussion and debate.
At 70, unlike the men and women who might have planted its seed, the banyan is still young. But without constant regeneration and expansion, it withers away. That could be the fate that awaits the United Nations. Either we can take this historic opportunity to renew this critical institution or we can condemn it to irrelevance and tragic withering. We could easily lose ourselves in a labyrinth of our own creation, but if we seize the day, we will see a United Nations growing to its full potential, a United Nations that can reduce the gap between what it does and what it is capable of doing, a mighty banyan tree providing the canopy for a peaceful and prosperous humankind. I end my statement with the hope that the United Nations will grow into such a banyan tree.
"""

# 1. FIT TOPIC MODEL TO TEXT:

In [ ]:
txt_chunks = f_make_text_chunks(txt_doc)

len(txt_chunks)

26

In [ ]:
kwargs = dict(n_neighs_umap = 3,
              n_dims_umap = 10,
              min_cluster_size = 3,
              dist_metric = 'euclidean')

%time topic_model, ner_model, topic_dict = f_fit_topic_model(txt_chunks, **kwargs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


CPU times: user 26.9 s, sys: 4.29 s, total: 31.2 s
Wall time: 33.8 s


# 2. CREATE TOPIC BASED CHROMADB COLLECTIONS:
(LOCAL VECTOR_DBs)

In [ ]:
embedding_function, character_splitter, token_splitter = f_chroma_instantiations()

In [ ]:
chroma_client = chromadb.Client()

f_create_chroma_collections(topic_dict, topic_model, chroma_client,
                            embedding_function, character_splitter, token_splitter)

collection 'topic_-1' created sucessfully
collection 'topic_0' created sucessfully
collection 'topic_1' created sucessfully
collection 'topic_2' created sucessfully
collection 'topic_3' created sucessfully


# 3. OPENAI INSTANTIATIONS:

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["HUGGINGFACE_API_KEY"] = userdata.get('HF_TOKEN')

openai_client = OpenAI()
llm = "gpt-3.5-turbo"

In [ ]:
prompt = """\

You are a multi talented helpful assistant, \
who is an experts in multiple fields, some of them being: \
social science, economics, finance, world affairs, law, artificial intelligence, \
physics, chemistry, biology, and more. \
You are known for answering questions by first thinking about the question deeply \
and then synthesizing relevant information from a collection of documents. \
your answers are articulate and to the point. \
You make intense use of the following principle: \
---Do not answer to anything not asked in the question--- \
You will be shown the user's question, and the relevant set of \
documents to derive and synthesize your answer from. \

Answer the user's question using ONLY these set of documents, please keep your answer \
limited to what is specifically being asked and do not talk about anything more. \
Make doubly sure that you are sticking to the facts from the documents provided and \
that your answer does not contain any information about anything that is \
not specifically mentioned in the question.
"""

# 4. GET TOPIC BASED OVERVIEW OF THE DOCUMENT:  
(GIVES US CONTEXT FOR ASKING QUESTIONS)

In [ ]:
f_display_topics(topic_dict)

TOPIC: -1
---------
('india, leaders, charter, banyan, revitalization, united, united nations, '
 'branches, mandates, peacekeeping, nations, tree')


TOPIC: 0
---------
('india, pakistan, terrorism, disaster, syria, states, terrorist, threat, '
 'menace, kashmir, terrorists, crisis')


TOPIC: 1
---------
('india, memorial, security council, united nations, peacekeepers, countries, '
 'solidarity, continents, peacekeeping, africa, anniversary, nations')


TOPIC: 2
---------
('sovereign, peace, reform, partnerships, goals, middle east, united, '
 'international, united nations, conflicts, palestinian, pacific island, '
 'nations')


TOPIC: 3
---------
('2030 agenda for sustainable development, india, nature, sabka, government, '
 'lifestyle, united nations, sustainable, modi, paris, development, minister, '
 'yoga')




# 6. QUERY ANY TOPIC:

### TOPIC 0:

In [ ]:
topic_n = 0
query = "Describe India's relationship with Pakistan?"

f_get_ans(query, topic_n, prompt, openai_client, llm)


ANSWER: 
--------
India's relationship with Pakistan is characterized by a stance of openness to dialogue but with a firm position that talks and terrorism cannot coexist. India has emphasized the need for Pakistan to give up supporting terrorism for any meaningful dialogue to take place. Despite efforts to address terrorism through talks and agreements such as the one reached in Ufa in July, there is a recognition of ongoing challenges. The issue of terrorism is a significant concern due to past terrorist attacks like the 2008 Mumbai attacks and recent cross-border incidents. These attacks are seen as attempts to destabilize India and support Pakistan's claims in the region.

Additionally, India acknowledges its role as a nuclear-weapon state and supports global nuclear disarmament efforts. It also highlights the importance of a coordinated international response to humanitarian crises, such as the refugee crisis in regions like Syria, Iraq, and Libya. India has been proactively invo

In [ ]:
topic_n = 0
query = "what does the document say about - 2030 agenda for sustainable development ?"

f_get_ans(query, topic_n, prompt, openai_client, llm)


ANSWER: 
--------
The document mentions that India is hosting the first Asian Ministerial Conference on Disaster Risk Reduction in November 2016, focusing on the Sendai Framework for Disaster Risk Reduction. The document also highlights India's stance on terrorism, emphasizing the importance of giving up terrorism for dialogue to address various issues effectively. Moreover, it stresses the need for international cooperation to combat global threats like epidemics and terrorism, advocating for a comprehensive convention on international terrorism to be unanimously adopted. Additionally, the document touches upon the challenges India faces in its relations with Pakistan, particularly regarding cross-border terrorism, the 2008 Mumbai terror attacks, and the need for Pakistan to act against terrorists operating from its territory.


### TOPIC 1:

In [ ]:
topic_n = 1
query = "what is being said about africa?"

f_get_ans(query, topic_n, prompt, openai_client, llm)


ANSWER: 
--------
The documents mention that Africa is a region with historical bonds with India, stemming from a common struggle against colonialism and a belief in shared prosperity. It is highlighted that India will host the third India-Africa Forum Summit, where leaders from all 54 African countries have been invited for the first time. The documents also reflect on the United Nations' peacekeeping efforts, where India has been a significant contributor with the largest deployment of peacekeepers. Additionally, tribute is paid to the peacekeepers, including those from India, who have made the ultimate sacrifice in the line of duty.


In [ ]:
topic_n = 1
query = "where have the peace keeping forces been deployed?"

f_get_ans(query, topic_n, prompt, openai_client, llm)


ANSWER: 
--------
The peacekeeping forces have been deployed in 10 missions, with about 8,000 Indian military and police personnel taking part in these operations. India has been a significant contributor to international security provided by the United Nations, with a total of 180,000 peacekeepers deployed so far. It is worth noting that more than 3,300 peacekeepers, including 161 from India, have made the ultimate sacrifice in these missions.


In [ ]:
topic_n = 1
query = "what is being said about united nations"

f_get_ans(query, topic_n, prompt, openai_client, llm)


ANSWER: 
--------
The United Nations is currently celebrating its seventieth anniversary, with the General Assembly being described as historic. India, as a signatory country of the United Nations charter, expresses its full support for the organization's efforts. India also has a strong historical connection with Africa, evident through the upcoming India-Africa forum summit. Additionally, India acknowledges the sacrifices made by peacekeepers, including those from India, and is committed to contributing to the United Nations peacekeepers memorial wall. The world is facing conflicts on multiple continents, and the need for political will to find alternatives to traditional solutions is emphasized, especially in peacekeeping efforts where India has been a significant contributor with a large number of military and police personnel deployed in challenging environments.


### TOPIC 2

In [ ]:
topic_n = 2
query = "what does the document say about the middle east?"

f_get_ans(query, topic_n, prompt, openai_client, llm)


ANSWER: 
--------
The document mentions that the country remains committed to the Middle East peace process as a key factor in preventing further radicalization in the region. It expresses hope for a solution to the Palestinian question and emphasizes the importance of engaging with the Middle East. It also highlights the need for reform within the United Nations, particularly in the Security Council, to better reflect the current global geopolitical landscape and include more developing nations in decision-making structures.


In [ ]:
topic_n = 2
query = "What does the document say about pacific island nations?"

f_get_ans(query, topic_n, prompt, openai_client, llm)


ANSWER: 
--------
The document mentions that efforts have been made to engage with Pacific Island nations as part of a broader outreach by the institution discussed in the document. Specifically, there was a summit held with 14 Pacific Island nations as a way to build modern economic partnerships. This engagement with Pacific Island nations is a part of a larger effort to strengthen relationships in the region.
